In [1]:
import torch
import torch.nn as nn

from functools import partial
from fastai import *
from fastai.basic_train import Learner

from keraTorch.data import create_db
from keraTorch.layers import *
# from keraTorch.loss import *

In [2]:
class Sequential:
    def __init__(self):
        self.layers = []
        self.last_dim = None
        
    def add(self, layer):
        layer = layer.get_layer(self.last_dim)
        self.last_dim = layer['output_dim']
        self.layers.extend(layer['layers'])
        
    def compile(self, loss, optimizer=None):
        self.model = nn.Sequential(self.layers)
        self.loss = loss
        
    def fit(self, x, y, bs, epochs, lr=1e-3, one_cycle=True, get_lr=True):
        db = create_db(x, y, bs)
        self.learn = Learner(db, self.model, loss_func=self.loss)
        if one_cycle:
            self.learn.fit_one_cycle(epochs, lr)
        else:
            self.learn.fit(epochs, lr)
               
    def lr_find(self, x, y):
        db = create_db(x, y)
        learn = Learner(db, self.model, loss_func=self.loss)
        learn.lr_find()
        learn.recorder.plot(suggestion=True)
        
    def predict(x):
        return self.learn.model(torch.Tensor(x))

In [3]:
from sklearn.datasets import load_iris
import numpy as np

In [4]:
data = load_iris()

In [5]:
x, y = data['data'], data['target']
x.shape, y.shape

((150, 4), (150,))

In [6]:
model = Sequential()

In [7]:
model.add(Dense(2, x.shape[1]))
model.add(Dense(2))
model.add(Dense(len(data['target_names'])))

TypeError: __init__() got an unexpected keyword argument 'input_dim'

In [ ]:
# %debug